In [1]:
import numpy as np
import shutil
from astropy.coordinates import Angle
from astropy.io import ascii, fits
from astropy import units as u 
from ants import absInt, fluxInt, headPlay, fitsPlay


abSex = absInt.absint()
fl = fluxInt.fluxint()
hp = headPlay.headplay()
fp = fitsPlay.fitsplay()


get_ipython().magic(u'pylab inline')


rootDir = '/Users/maccagni/Projects/MFS/FornaxA/centreHI/'
cubeDir = '/Users/maccagni/data/MFS/MeerKATComm/fa4/cubes/'
momDir = rootDir+'FA_centre/'
mom0Name= momDir+'MIAPP-t10_6_mom0.fits'


print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''

Populating the interactive namespace from numpy and matplotlib
	+---------+
	 Inputs loaded
	+---------+


### Convert mom0 Map to column density

$$
N_{\rm HI} = 3.1\times10^{17} \frac{SdV}{\Theta^2}
$$


- SdV : integrated flux in mJy*km/s or Jy * m/s 
- $\Theta^2$ : beam size in arcmin 

In [5]:
momHead = hp.printHead(mom0Name)

bMaj = Angle(momHead['BMAJ'], u.deg)
bMin = Angle(momHead['BMIN'], u.deg)

conversionFactor = 3.1e17/(bMaj.arcminute*bMin.arcminute)

nHIName=momDir+'mom0NHI.fits'

fp.multVal(mom0Name,conversionFactor,nHIName)

print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''

SIMPLE 	True
BITPIX 	-32
NAXIS 	2
NAXIS1 	25
NAXIS2 	34
CTYPE1 	RA---SIN
CRPIX1 	14.0
CDELT1 	-0.00138888888889
CRVAL1 	50.67375
CTYPE2 	DEC--SIN
CRPIX2 	18.0
CDELT2 	0.00138888888889
CRVAL2 	-37.2083333333
ORIGIN 	SoFiA 2.0.0
SPECSYS 	0.0
EQUINOX 	2000.0
BUNIT 	Jy/beam*m/s
BMAJ 	0.00664442496252
BMIN 	0.00533499983768
BPA 	134.940537921
23.9199298651
0.398665497751
/Users/maccagni/Projects/MFS/FornaxA/centreHI/FA_centre/mom0NHI.fits
	+---------+
	 Inputs loaded
	+---------+


In [16]:
coName = rootDir+'cubes/CO_cube_orig.fits'
coNameOut = rootDir+'cubes/CO_cube_orig_vrad.fits'

coHead = hp.printHead(coName)


SIMPLE 	True
BITPIX 	-32
NAXIS 	3
NAXIS1 	80
NAXIS2 	90
NAXIS3 	490
EXTEND 	True
BSCALE 	1.0
BZERO 	0.0
BMAJ 	0.00421535078806
BMIN 	0.00214837862369
BPA 	-89.3305261085
BTYPE 	Intensity
OBJECT 	NGC1316
BUNIT 	Jy/beam
RADESYS 	ICRS
LONPOLE 	180.0
LATPOLE 	-37.2081998736
PC1_1 	1.0
PC2_1 	0.0
PC3_1 	0.0
PC1_2 	0.0
PC2_2 	1.0
PC3_2 	0.0
PC1_3 	0.0
PC2_3 	0.0
PC3_3 	1.0
CTYPE1 	RA---SIN
CRVAL1 	50.6741208333
CDELT1 	-0.000555555555556
CRPIX1 	40.0
CUNIT1 	deg
CTYPE2 	DEC--SIN
CRVAL2 	-37.2081998736
CDELT2 	0.000555555555556
CRPIX2 	46.0
CUNIT2 	deg
CTYPE3 	FREQ
CRVAL3 	1.137664914e+11
CDELT3 	3824570.0
CRPIX3 	1.0
CUNIT3 	Hz
PV2_1 	0.0
PV2_2 	0.0
RESTFRQ 	1.15271202e+11
SPECSYS 	LSRK
ALTRVAL 	3913387.57231
ALTRPIX 	1.0
VELREF 	257
COMMENT 	casacore non-standard usage: 4 LSD, 5 GEO, 6 SOU, 7 GAL
TELESCOP 	ALMA
OBSERVER 	kmatsui
DATE-OBS 	2017-10-16T07:11:21.887999
TIMESYS 	UTC
OBSRA 	50.6741208333
OBSDEC 	-37.2081998736
OBSGEO-X 	2225142.18027
OBSGEO-Y 	-5440307.37035
OBSGEO-Z 	-2481029.85

In [17]:
vel = (coHead['RESTFRQ']-coHead['CRVAL3'])/coHead['RESTFRQ']
velStep = (coHead['RESTFRQ']-(coHead['CRVAL3']+coHead['CDELT3']))/coHead['RESTFRQ']
velStep = velStep - vel

coHead['CRVAL3'] = vel*2.99792458e8
coHead['CDELT3'] = velStep*2.99792458e8

coHead['CTYPE3'] = 'VRAD'


f= fits.open(coName)
datas=f[0].data

fits.writeto(coNameOut,datas,coHead,overwrite=True)
